In [1]:
import datetime
import re
import os
import numpy as np
import matplotlib.pyplot as plt
import filament
import logging
import copy
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.mpl.ticker as cartopyticker
myproj = ccrs.PlateCarree()
coast = cfeature.GSHHSFeature(scale="f")
from osgeo import gdal, osr
import logging
from importlib import reload

In [2]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.info("Starting")

INFO:root:Starting


## Input parameters
We set the date of interest and the domain coordinates are taken from the geoTIFF file.

In [3]:
year, month, day = 2019, 5, 1
figdir = "../figures/wind/"
visibledir = "/data/Visible/Balearic/"
os.path.exists(figdir)

True

## Data

In [4]:
datewind = datetime.datetime(year, month, day)
dayofyear = datewind.timetuple().tm_yday

### Visible image

In [5]:
visibleim = datewind.strftime("VIIRS-%Y-%m-%dT00_00_00Z.tiff")
#visibleim = datewind.strftime("Sentinel-3_OLCI_%Y-%m-%d.tiff")
#visibleim = datewind.strftime("Sentinel-2_L1C_%Y-%m-%d.tiff")
#visibleim = datewind.strftime("Terra-%Y-%m-%dT00_00_00Z.tiff")
if os.path.exists(os.path.join(visibledir, visibleim)):
    logger.info("Reading geoTIFF file {}".format(visibleim))
    vis = filament.Visible()
    vis.read_geotiff(os.path.join(visibledir, visibleim))
    extent2 = copy.deepcopy(vis.extent)
    extent2[0] -= 360.
    extent2[1] -= 360.
else:
    extent2 = [-360.551513671875, -356.715087890625, 36.77124023437501, 40.86694335937501]
    extent2 = [-(360 - 2.), -(360. - 10.), 36.5, 45.]
    logger.info("No visible image for that date")
domain = [2., 10., 36.5, 45.]

INFO:root:No visible image for that date


### Scatterometer wind
The wind (netCDF) are read from the JPL OPEnDAP server.      
The files are sorted by years and days of year.

With these pieces of information we can create two lists of files (one for MetOp-A, one for MetOp-A).

In [6]:
filelist1 = filament.get_filelist_url(year, dayofyear)
# filelist1 = ["https://opendap.jpl.nasa.gov:443/opendap/OceanWinds/ascat/preview/L2/metop_b/coastal_opt/2019/332/ascat_20191128_092400_metopb_37332_eps_o_coa_3202_ovw.l2.nc.gz"]

INFO:Filament:Found 28 files


In [ ]:
#filelist1 = ["/data/Wind/ASCAT/WestMed/ascat_20200401_201500_metopa_69801_eps_o_coa_3202_ovw.l2.nc.gz"]

In [ ]:
filelistclean = []
 
for ifile, dataurl in enumerate(filelist1):

    logger.info("Working on {}".format(dataurl))
    wind = filament.Wind()
    res = wind.read_ascat(dataurl, extent2)

    if res:    
        filelistclean.append(dataurl)
        mm = str(month).zfill(2)
        dd = str(day).zfill(2)
        
        fname = "".join(("Wind-", dataurl.split("/")[-1].split(".")[0]))
        logger.info("Saving figure as {}".format(fname))
        figname = os.path.join(figdir, fname)
        
        fig = plt.figure(figsize=(8, 8))
        ax = plt.subplot(111, projection=myproj)
        # vis.add_to_plot(ax, myproj)

        wind.add_to_plot(fig, ax, domain=domain,
                         cmap=plt.cm.hot_r, clim=[0, 15], quivscale=300, quivwidth=0.002,
                         cbarloc='upper right', 
                         date="{}-{}-{}".format(year, mm, dd))
        
        filament.decorate_map(ax, 
                                 domain, 
                                 np.arange(0.0, 10., 1.0), 
                                 np.arange(36., 46., 1.0))
        
        ax.add_feature(coast, linewidth=.2, color=".7")
        
        plt.savefig(figname, dpi=300, bbox_inches="tight")
        # plt.show()
        plt.close()

INFO:root:Working on https://opendap.jpl.nasa.gov:443/opendap/OceanWinds/ascat/preview/L2/metop_a/coastal_opt/2019/121/ascat_20190501_005100_metopa_65016_eps_o_coa_3201_ovw.l2.nc.gz
INFO:Filament:No data in the region of interest
INFO:root:Working on https://opendap.jpl.nasa.gov:443/opendap/OceanWinds/ascat/preview/L2/metop_a/coastal_opt/2019/121/ascat_20190501_023000_metopa_65017_eps_o_coa_3201_ovw.l2.nc.gz
INFO:Filament:No data in the region of interest
INFO:root:Working on https://opendap.jpl.nasa.gov:443/opendap/OceanWinds/ascat/preview/L2/metop_a/coastal_opt/2019/121/ascat_20190501_041200_metopa_65018_eps_o_coa_3201_ovw.l2.nc.gz
INFO:Filament:No data in the region of interest
INFO:root:Working on https://opendap.jpl.nasa.gov:443/opendap/OceanWinds/ascat/preview/L2/metop_a/coastal_opt/2019/121/ascat_20190501_055400_metopa_65019_eps_o_coa_3201_ovw.l2.nc.gz
INFO:Filament:No data in the region of interest
INFO:root:Working on https://opendap.jpl.nasa.gov:443/opendap/OceanWinds/ascat/p